# Carleton University COMP 3105 MNIST Demo
Data
- Files on the left menu -> sample_data

GPU Runtime (faster)
- Runtime at the top menu -> Change runtime type -> Hardware Accelerator: GPU

The code here are adapted from
https://github.com/pytorch/examples/tree/main/mnist

In [ ]:
!head sample_data/mnist_train_small.csv

6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,67,67,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,131,252,252,66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,159,250,232,30,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,222,252,108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147,252,183,5,0,0,0,0,0,0,0,20,89,89,73,0,0,0,0,0,0,0,0,0,0,0,0,48,247,252,159,0,0,0,0,0,0,0,79,236,252,252,249,198,16,0,0,0,0,0,0,0,0,0,41,193,252,199,22,0,0,0,0,0,12,135,248,252,252,252,252,252,100,0,0,0,0,0,0,0,0,0,100,252,252,88,0,0,0,0,0,11,171,252,252,235,175,178,252,252,224,0,0,0,0,0,0,0,0,15,209,252,233,12,0,0,0,0,49,177,252,252,89,26,0,2,166,252,252,0,0,0,0,0,0,0,0,96,253,253,59,0,0,0,0,11,177,255,253,92,0,0,0,0,155,253,128,0,0,0,0,0,0,0,0,143,252,252,10,0,0,0,12,171,252,216,110,13,0,0,0,3,180,

In [ ]:
!wc -l sample_data/mnist_train_small.csv
!wc -l sample_data/mnist_test.csv

20000 sample_data/mnist_train_small.csv
10000 sample_data/mnist_test.csv


In [1]:
import time

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Prepare dataset
def prepareData(file_name):
    train_data = np.loadtxt(file_name, delimiter=',')
    y = train_data[:, 0]
    X = train_data[:, 1:] / 255.

    # Convert to PyTorch Tensors
    tensor_X = torch.Tensor(X)
    tensor_y = torch.LongTensor(y)

    dataset = TensorDataset(tensor_X, tensor_y) # create PyTorch TensorDataset
    dataloader = DataLoader(dataset, batch_size=32) # create PyTorch DataLoader

    return dataset, dataloader

train_dataset, train_dataloader = prepareData('sample_data/mnist_train_small.csv')
test_dataset, test_dataloader = prepareData('sample_data/mnist_test.csv')

In [3]:
# Define neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)  # fully connected layers
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.25)  # PyTorch: drop probability

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)  # ReLU activation
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)  # softmax for multi-class classification
        return output

device = "cuda"
model = Net().to(device)  # use GPU

In [4]:
# Optimizer: Stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
# Training loop
def train(model, optimizer, train_dataloader, device):
    model.train()  # entering training mode (dropout behaves differently)
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.to(device), target.to(device)  # move data to the same device
        optimizer.zero_grad()  # clear existing gradients
        output = model(data)  # forward pass
        loss = F.nll_loss(output, target)  # compute the loss
        loss.backward()  # backward pass: calculate the gradients
        optimizer.step()  # take a gradient step


In [6]:
# Test loop
def test(model, test_dataloader, device):
    model.eval()  # entering evaluation mode (dropout behaves differently)
    m = len(test_dataloader.dataset)
    test_loss = 0
    correct = 0
    with torch.no_grad():  # do not compute gradient
        for data, target in test_dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= m
    accuracy = correct / m
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.3f}')
    return test_loss, accuracy


In [7]:
# Training
n_epochs = 30
start_time = time.time()
for epoch in range(n_epochs):
    train(model, optimizer, train_dataloader, device)
    test(model, test_dataloader, device)
print(f"Time: {time.time() - start_time}")


Test set: Average loss: 1.5049, Accuracy: 0.679
Test set: Average loss: 0.5997, Accuracy: 0.840
Test set: Average loss: 0.4392, Accuracy: 0.879
Test set: Average loss: 0.3721, Accuracy: 0.895
Test set: Average loss: 0.3341, Accuracy: 0.905
Test set: Average loss: 0.3084, Accuracy: 0.912
Test set: Average loss: 0.2890, Accuracy: 0.916
Test set: Average loss: 0.2720, Accuracy: 0.921
Test set: Average loss: 0.2571, Accuracy: 0.926
Test set: Average loss: 0.2453, Accuracy: 0.929
Test set: Average loss: 0.2336, Accuracy: 0.932
Test set: Average loss: 0.2230, Accuracy: 0.935
Test set: Average loss: 0.2126, Accuracy: 0.938
Test set: Average loss: 0.2057, Accuracy: 0.939
Test set: Average loss: 0.1966, Accuracy: 0.942
Test set: Average loss: 0.1889, Accuracy: 0.944
Test set: Average loss: 0.1840, Accuracy: 0.945
Test set: Average loss: 0.1768, Accuracy: 0.947
Test set: Average loss: 0.1722, Accuracy: 0.948
Test set: Average loss: 0.1666, Accuracy: 0.950
Test set: Average loss: 0.1609, Accuracy